In [27]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your api key here"

In [28]:
!pip install langchain chromadb faiss-cpu tiktoken langchain_community langchain-community wikipedia

In [29]:
!pip install langchain_huggingface

In [30]:
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain.vectorstores import Chroma

#### Wikipedia Retrievers

In [31]:
from langchain.retrievers import WikipediaRetriever

In [32]:
# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [33]:
# Define your query
query = "the geopolitical history of india and pakistan from the perspective of a chinese"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [34]:
docs

[Document(metadata={'title': 'History of Pakistan (1947–present)', 'summary': "The history of the Islamic Republic of Pakistan began on 14 August 1947 when the country came into being in the form of the Dominion of Pakistan within the British Commonwealth as the result of the Pakistan Movement and the partition of India. While the history of the Pakistani nation according to the Pakistan government's official chronology started with the Islamic rule over Indian subcontinent by Muhammad ibn al-Qasim which reached its zenith during Mughal era. In 1947, Pakistan consisted of West Pakistan (today's Pakistan) and East Pakistan (today's Bangladesh). The President of All-India Muslim League and later the Pakistan Muslim League, Muhammad Ali Jinnah became Governor-General while the secretary general of the Muslim League, Liaquat Ali Khan became Prime Minister. The constitution of 1956 made Pakistan an Islamic democratic country.\nPakistan faced a civil war and Indian military intervention in 1

In [35]:
# print the retrieved docs in content

for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(f"Content:\n{doc.page_content}...")  # truncate for display
    print("\n")

Document 1:
Content:
The history of the Islamic Republic of Pakistan began on 14 August 1947 when the country came into being in the form of the Dominion of Pakistan within the British Commonwealth as the result of the Pakistan Movement and the partition of India. While the history of the Pakistani nation according to the Pakistan government's official chronology started with the Islamic rule over Indian subcontinent by Muhammad ibn al-Qasim which reached its zenith during Mughal era. In 1947, Pakistan consisted of West Pakistan (today's Pakistan) and East Pakistan (today's Bangladesh). The President of All-India Muslim League and later the Pakistan Muslim League, Muhammad Ali Jinnah became Governor-General while the secretary general of the Muslim League, Liaquat Ali Khan became Prime Minister. The constitution of 1956 made Pakistan an Islamic democratic country.
Pakistan faced a civil war and Indian military intervention in 1971 resulting in the secession of East Pakistan as the new 

#### Vector Store Retriever

In [36]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_core.documents import Document

In [37]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [39]:
# Step 2: Initialize embedding model
embedding_model = HuggingFaceEndpointEmbeddings(model = "sentence-transformers/all-MiniLM-L6-v2")

# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [40]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [41]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [42]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Chroma is a vector database optimized for LLM-based search.


In [43]:
results = vectorstore.similarity_search(query, k=2)

In [44]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Chroma is a vector database optimized for LLM-based search.


In [45]:
# if similarity search is doing the same thing then why do we need vector store as retrievers
# because it uses just the sililarity search for searching technique while using it as retrievers
# gives us libiality to use different search teachnique

#### MMR : Maximum Marginal Relevance


In [46]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [47]:
from langchain_community.vectorstores import FAISS

In [48]:
# Initialize OpenAI embeddings
embedding_model = HuggingFaceEndpointEmbeddings(model = "sentence-transformers/all-MiniLM-L6-v2")

# Step 2: Create the FAISS vector store from documents
vectorestore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [49]:
# enable MMR in the Retriever

retriever = vectorestore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [50]:
query = "What is langchain?"
results = retriever.invoke(query)

In [51]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 2 ---
LangChain is used to build LLM based applications.

--- Result 3 ---
Embeddings are vector representations of text.


#### Multiquery Retriever

In [52]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_core.documents import Document
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.retrievers.multi_query import MultiQueryRetriever

In [53]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [55]:
# Initialize Huggingface embeddings
embedding_model = HuggingFaceEndpointEmbeddings(model = "sentence-transformers/all-MiniLM-L6-v2")

In [56]:
#  create a FAISS vector store
vectorstore = FAISS.from_documents(documents= all_docs, embedding=embedding_model)

In [57]:
# Create a Retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [74]:
# create a llm
llm = HuggingFaceEndpoint(
    repo_id='mistralai/Mistral-7B-Instruct-v0.2',
    task="chat-completion",
)

In [76]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatHuggingFace(llm = llm)
)

In [77]:
# Query
query = "How to improve energy levels and maintain balance?"

In [78]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [79]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 5 ---
Photosynthesis enables plants to produce energy by converting sunlight.
******************************************************************************************************************************************************

--- Result 1 ---
Photosynthesis enables plants to produce energy by converting sunlight.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 4 ---
Black holes bend spacetime and store immense gravitational energy.

---

#### ContextualCompressionRetriever

In [80]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import  ChatHuggingFace, HuggingFaceEndpoint
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [81]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [ ]:
# Create a FAISS vector store from the documents
embedding_model = HuggingFaceEndpointEmbeddings(model= "sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding_model)

In [86]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [87]:
# Set up the compressor using an LLM

llm = HuggingFaceEndpoint(
    repo_id='mistralai/Mistral-7B-Instruct-v0.2',
    task="chat-completion",
)
llm_model = ChatHuggingFace(llm = llm)

compressor = LLMChainExtractor.from_llm(llm_model)

In [88]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [89]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [90]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis enables plants to produce energy by converting sunlight.

--- Result 2 ---
NO_OUTPUT. The context does not provide any information relevant to answering the question about photosynthesis.

--- Result 3 ---
NO_OUTPUT. The context does not contain any information relevant to the question about photosynthesis.

--- Result 4 ---
NO_OUTPUT. The question is about photosynthesis, which is a process that occurs in green plants and certain bacteria, involving the absorption of light energy and conversion into chemical energy through a series of reactions. None of the context provided is related to this topic.

--- Result 5 ---
NO_OUTPUT. The context provided does not contain any information relevant to the question about photosynthesis.
